<a href="https://colab.research.google.com/github/dansojo/Medical_CV/blob/main/part2_mask%EC%A0%81%EC%9A%A9_%EC%9D%B4%EB%AF%B8%EC%A7%80%EC%83%9D%EC%84%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import cv2
import numpy as np

# 경로 설정
ORIGINAL_IMAGE_DIR = "/content/drive/MyDrive/Medical_CV/피부암 분류 및 Segmentation/part2"
MASK_IMAGE_DIR = "/content/drive/MyDrive/Medical_CV/피부암 분류 및 Segmentation/HAM10000_segmentations_lesion_tschandl"
SAVE_DIR = "/content/drive/MyDrive/Medical_CV/피부암 분류 및 Segmentation/part2 + mask이미지"

# 저장 폴더 생성
os.makedirs(SAVE_DIR, exist_ok=True)

In [ ]:
# 병변 강조 이미지 생성 및 저장 함수
def create_emphasized_images(image_dir, mask_dir, save_dir):
    # 저장 경로 생성
    os.makedirs(save_dir, exist_ok=True)

    # Part2 이미지 파일 리스트 (확장자 제거)
    image_files = [f for f in os.listdir(image_dir) if f.endswith(".jpg")]
    image_names = [os.path.splitext(f)[0] for f in image_files]  # 확장자 제거한 이름

    print(f"Part2 이미지 개수: {len(image_names)}")

    # 마스크 매칭 및 처리
    for image_name in image_names:
        image_path = os.path.join(image_dir, image_name + ".jpg")
        mask_path = os.path.join(mask_dir, image_name + "_segmentation.png")

        # 이미지와 마스크 존재 확인
        if not os.path.exists(mask_path):
            print(f"마스크 없음, 건너뜀: {image_name}")
            continue

        # 이미지와 마스크 로드
        image = cv2.imread(image_path)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)  # 마스크는 Grayscale

        if image is None or mask is None:
            print(f"로드 실패, 건너뜀: {image_name}")
            continue

        # 병변 강조 (마스크와 이미지 곱하기)
        mask_binary = (mask > 0).astype(np.uint8)  # 이진 마스크 생성 (0, 1)
        emphasized_image = cv2.bitwise_and(image, image, mask=mask_binary)

        # 배경 어둡게 하기 (옵션)
        background = cv2.addWeighted(image, 0.3, np.zeros_like(image), 0.7, 0)
        final_image = cv2.add(emphasized_image, background)

        # 명암 대비 조정 (옵션)
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        for c in range(3):  # RGB 채널별로 적용
            final_image[..., c] = clahe.apply(final_image[..., c])

        # 저장
        save_path = os.path.join(save_dir, image_name + ".jpg")
        cv2.imwrite(save_path, final_image)

        print(f"Saved emphasized image: {save_path}")

# 실행
create_emphasized_images(ORIGINAL_IMAGE_DIR, MASK_IMAGE_DIR, SAVE_DIR)
